In [97]:
import numpy as np
from scipy.stats import multivariate_normal as mv

*N.B. All of the expectations below are conditional on x*


Taste variation:  $ \beta_n \sim \mathcal{N}(b, \sigma^2_{\beta})$. This model specification allows to estimate $b, \sigma^2_{\beta}$

Each agent's utility is:
$$ \begin{cases} U_{n1} = \beta_{n}'x_{n1} + \varepsilon_{n1} \\ U_{n2} = \beta_{n}'x_{n2} + \varepsilon_{n2} \end{cases} $$

where $\varepsilon_n = (\varepsilon_{n1}, \varepsilon_{n2}) \sim \mathcal{N}(\pmb{0}, \pmb{\Sigma})$

*Train* suggests to rewrite it as follows:
 $ U_{nj} = b'x_{nj} + \tilde{\beta}_n'x_{nj} + \varepsilon_{nj} $, where $\tilde{\beta}_n = \beta_n - b$. Let $ \eta_{nj} =\tilde{\beta}_n'x_{nj} + \varepsilon_{nj} $ and we obtain new system of utilities:

$$ \begin{cases} U_{n1} = \beta_{n}'x_{n1} + \varepsilon_{n1} \\ U_{n2} = \beta_{n}'x_{n2} + \varepsilon_{n2} \end{cases} \to \begin{cases} U_{n1} = b'x_{n1} + \eta_{n1} \\ U_{n2} = b'x_{n2} + \eta_{n2} \end{cases}$$

and it is easy to show that conditional on data $$ \mathbb{E} (\eta_{nj}) = \mathbb{E}(\tilde{\beta}_n'x_{nj} + \varepsilon_{nj}) = 0  $$
what follows from the error iid and the beta's distribution. The matrix of covariances for errors with imposed restrictions on the properties of utility functions has the following form:

$$ \text{Cov}(\eta_{n1}, \eta_{n2}) = \Omega = \sigma^2_{\beta} \begin{pmatrix} x^2_{n1}& x_{n1}x_{n2} \\ x_{n1}x_{n2}& x^2_{n2}  \end{pmatrix} + \begin{pmatrix}1& 0 \\0&1\end{pmatrix}$$


$$ \eta_{n} = (\eta_{n1}, \eta_{n2}) \sim \mathcal{N}\left( \begin{pmatrix} 0 \\0  \end{pmatrix}, \begin{pmatrix} \sigma^2_{\beta}x^2_{n1} + 1& \sigma^2_{\beta}x_{n1}x_{n2} \\ \sigma^2_{\beta}x_{n1}x_{n2}& \sigma^2_{\beta}x^2_{n2} + 1  \end{pmatrix} \right)$$
which implies that

$$ P(U_{n1} > U_{n2}) = P(b'x_{n1} + \eta_{n1} >  b'x_{n2} + \eta_{n2}) = P(\eta_{n1}-\eta_{n2} >  b'x_{n2} - b'x_{n1} ) = 1 - P(\eta_{n1}-\eta_{n2} \le  b'x_{n2} - b'x_{n1} ) $$

and finally finding distribution for $\eta_{n1}- \eta_{n2}$: difference is distributed normally as linear combination of normal variables. Expectation is:
$$ \mathbb{E}(\eta_{n1}- \eta_{n2}) = \mathbb{E}(\eta_{n1})- \mathbb{E}(\eta_{n2}) = 0 $$
and the variance is just:
$$ \mathbb{V}(\eta_{n1}- \eta_{n2}) = \mathbb{V}(\eta_{n1}) + \mathbb{V}(\eta_{n2}) - 2\text{Cov}(\eta_{n1}, \eta_{n2}) = \sigma^2_{\beta}x^2_{n1} + 1 + \sigma^2_{\beta}x^2_{n2} + 1 - 2\sigma^2_{\beta}x_{n1}x_{n2} = 2 + \sigma^2_{\beta}(x_{n1} - x_{n2})^2 $$

Then it immediately follows that:
$$ \eta_{n1}- \eta_{n2} \sim \mathcal{N} \left( 0, 2 + \sigma^2_{\beta}(x_{n1} - x_{n2})^2\right) = \sqrt{2 + \sigma^2_{\beta}(x_{n1} - x_{n2})^2} \times\mathcal{N}(0, 1)$$

Getting back to $P(U_{n1} > U_{n2})$ yields:
$$ P(U_{n1} > U_{n2}) = 1 - P(\eta_{n1}-\eta_{n2} \le  b'x_{n2} - b'x_{n1} ) = 1 - \Phi\left(\frac{b'x_{n2} - b'x_{n1}}{\sqrt{2 + \sigma^2_{\beta}(x_{n1} - x_{n2})^2}}\right) $$

We will estimate the parameters $b, \sigma^2_{\beta}$ using MLE approach. The likelihood function for person $n$ is just:

$$ \mathcal{L}_n(b, \sigma^2_{\beta}) = \prod_i P(U_{n1i} > U_{n2i})^{y_i}P(U_{n2i} > U_{n1i})^{1-y_i}$$
and log-likelihood is:

$$ l_n(b, \sigma^2_{\beta}) = \sum_i \left( y_i \ln(P(U_{n1i} > U_{n2i}))  + (1-y_i)\ln(P(U_{n2i} > U_{n1i}))  \right) $$
Inserting formula we obtained above yields:

$$ l_n(b, \sigma^2_{\beta}) =\sum_i \left[ y_i \ln\left(1 - \Phi\left(\frac{b'x_{n2i} - b'x_{n1i}}{\sqrt{2 + \sigma^2_{\beta}(x_{n1i} - x_{n2i})^2}}\right)\right)  + (1-y_i)\ln\left( \Phi\left(\frac{b'x_{n2i} - b'x_{n1i}}{\sqrt{2 + \sigma^2_{\beta}(x_{n1i} - x_{n2i})^2}}\right)  \right)  \right] \to \max_{b, \sigma^2_{\beta}}$$


The only thing left is to account for all agents $n = 1..N$

### Simulation (to-do)

In [18]:
X = np.random.normal(loc=1, scale=3, size=(100, 2))
e = np.random.normal(size=(100, 2))
beta = np.random.normal(loc=0.5, scale=0.75, size=(100, 1))
U = beta * X + e
y = np.argmax(U, axis=1)

In [103]:
def get_cov(sigma):
    global X
    return sigma * np.cov(X, rowvar=False) + np.ones(2)

def get_pdf(x, sigma):
    global X
    return mv(mean=[0, 0], cov=get_cov(sigma)).pdf(x)    

In [ ]:
def likelihood():
    pass

In [ ]:
eta = np.random.normal()